In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.stats as st
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import hamming_loss
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from imblearn.over_sampling import SMOTE 
from sklearn.cluster import KMeans

In [5]:
# https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29#
frogs_data = pd.read_csv('../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv')
X = frogs_data.drop(['Family', 'Genus', 'Species', 'RecordID'], axis=1)
y = frogs_data[['Family', 'Genus', 'Species']]

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_train

,Family,Genus,Species
2004,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1194,Dendrobatidae,Ameerega,Ameeregatrivittata
5359,Hylidae,Hypsiboas,HypsiboasCinerascens
1756,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
497,Leptodactylidae,Adenomera,AdenomeraAndre
...,...,...,...
3772,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
5191,Hylidae,Hypsiboas,HypsiboasCinerascens
5226,Hylidae,Hypsiboas,HypsiboasCinerascens
5390,Hylidae,Hypsiboas,HypsiboasCinerascens


###### Each instance has three labels: Families, Genus, and Species. Each of the labels has multiple classes. We wish to solve a multi-class and multi-label problem. One of the most important approaches to multi-label classification is to train a classifier for each label (binary relevance).

# 1
## (b)
### i.

https://mmuratarat.github.io/2020-01-25/multilabel_classification_metrics

https://en.wikipedia.org/wiki/Multi-label_classification#Statistics_and_evaluation_metrics
###### <ins>Evaluating Multilabel Classification</ins>
- **Exact Match Ratio**$ = \frac{1}{n}\sum_i^n{ I(y_i = \hat{y}_i})$
    - this metric is the ratio of correctly classed observations and total observations
    - It makes no distinction between predictions that are partially correct and completely incorrect
- **Hamming Loss** = $\frac{1}{n}\sum_i^n{\sum_j^L{\frac{1}{L}(I(y_i^j≠\hat{y}_i^j}})$
    - The inner summation is number of labels that were identified incorrectly
        - meaning if an observation has 5 labels and you correctly identified 2, the hamming score is .6
    - the outer summation averages the hamming score over all observations
    - Range: [0, 1] where 0 is the optimal value
- **Jaccard Index** = $\frac{Y ∩ \hat{Y}}{Y ∪ \hat{Y}} = \frac{TP}{TP+TN+FN}$
    - the number of correctly predicted labels divided by the sum of the number of true labels and the number of predicted labels
    - Thus, the best value is where $Y$ and $\hat{Y}$ are the same, meaning the Jaccard Index is .5

### i. <ins>Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers</ins>

In [6]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
# https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
# https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html

gammas = [.001, .01, 1] # kernel widths
penalties = [1, 10, 100, 1000, 10000] # penalty term scalars
ham_c_gamma_df = pd.DataFrame(index=gammas, columns=penalties) # holds hamming loss for each gamma, C pair
exact_c_gamma_df = pd.DataFrame(index=gammas, columns=penalties) # holds exact match score for each gamma, C pair
kf = KFold(n_splits=10, shuffle=True, random_state=49)

for c in penalties: # test indicated penalty scalars
    avg_hams = []
    avg_exact = []
    for gamma in gammas: # test indicated kernel widths
        
        # 10-fold CV
        for train_index, test_index in kf.split(X_train):
            X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
            y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]
            
            hamming_scores = []
            exact_scores = []
            pred_df = pd.DataFrame(columns=y_train.columns, index=y_test_cv.index)
            
            # train SVM model for each output-label-column
            for col in y_train.columns:
                pipe = Pipeline(steps= [('scale', StandardScaler()), ('svm', SVC(gamma=gamma, kernel='rbf', C=c))])
                ovr = OneVsRestClassifier(pipe, n_jobs=-1).fit(X_train_cv, y_train_cv[col])
                
                pred_df[col] = ovr.predict(X_test_cv)
            
            # calculate hamming loss
            for i in y_test_cv.index:
                ham_score = 0
                if pred_df['Family'][i] == y_test_cv['Family'][i]:
                    ham_score += 1
                if pred_df['Genus'][i] == y_test_cv['Genus'][i]:
                    ham_score += 1
                if pred_df['Species'][i] == y_test_cv['Species'][i]:
                    ham_score += 1
                
                exact_scores.append(1 if ham_score == 3 else 0)
                hamming_scores.append(1 - (ham_score / 3))
                
        avg_exact.append(sum(exact_scores) / len(exact_scores))
        avg_hams.append(sum(hamming_scores) / len(hamming_scores))
        
    ham_c_gamma_df[c] = avg_hams
    exact_c_gamma_df[c] = avg_exact

print('Cross Validated Hamming Scores: ')
print(ham_c_gamma_df)
print('\nCross Validated Exact Match Scores: ')
print(exact_c_gamma_df)

Cross Validated Hamming Scores: 
          1         10        100       1000      10000
0.001  0.091451  0.059642  0.034460  0.012591  0.011266
0.010  0.042412  0.013254  0.007290  0.008615  0.008615
1.000  0.098741  0.088138  0.088138  0.088138  0.088138

Cross Validated Exact Match Scores: 
          1         10        100       1000      10000
0.001  0.878728  0.910537  0.954274  0.982107  0.982107
0.010  0.946322  0.978131  0.988072  0.990060  0.990060
1.000  0.878728  0.888668  0.888668  0.888668  0.888668


##### The Exact Match score maxes out when gamma = .01 and the penalty scalar is ≥ 1000. Interestingly, the Hamming loss prefers a smaller penalty and is lowest when C = 100.  

Note that hamming score is just 1 - hamming loss (as per TA's comment on piazza post @1193)

In [7]:
# hamming score
ham_c_gamma_df.applymap(lambda x: 1-x)

,1,10,100,1000,10000
0.001,0.908549,0.940358,0.965540,0.987409,0.988734
0.010,0.957588,0.986746,0.992710,0.991385,0.991385
1.000,0.901259,0.911862,0.911862,0.911862,0.911862


### iii. <ins>Re-train SVM with L1-penalty</ins>

In [8]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html
penalties = [10, 100, 1000, 10000] # penalty term scalars
ham_c_df = pd.DataFrame(columns=penalties) # holds hamming loss for each gamma, C pair
exact_c_df = pd.DataFrame(columns=penalties) # holds exact match score for each gamma, C pair
kf = KFold(n_splits=10, random_state= 49, shuffle=True)

for c in penalties: # test indicated penalty scalars
    avg_hams = []
    avg_exact = []
        
    # 10-fold CV
    for train_index, test_index in kf.split(X_train):
        X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

        hamming_scores = []
        exact_scores = []
        pred_df = pd.DataFrame(columns=y_train.columns, index=y_test_cv.index)

        # train SVM model for each output-label-column
        for col in y_train.columns:
            pipe = Pipeline(steps= [('scale', StandardScaler()), 
                                    ('svm', LinearSVC(penalty='l1', multi_class='ovr', max_iter=10000,
                                                      C=c, loss='squared_hinge', dual=False))])
            ovr = pipe.fit(X_train_cv, y_train_cv[col])

            pred_df[col] = ovr.predict(X_test_cv)

        # calculate hamming loss
        for i in y_test_cv.index:
            ham_score = 0
            if pred_df['Family'][i] == y_test_cv['Family'][i]:
                ham_score += 1
            if pred_df['Genus'][i] == y_test_cv['Genus'][i]:
                ham_score += 1
            if pred_df['Species'][i] == y_test_cv['Species'][i]:
                ham_score += 1

            exact_scores.append(1 if ham_score == 3 else 0)
            hamming_scores.append(1 - (ham_score / 3))

    avg_exact.append(sum(exact_scores) / len(exact_scores))
    avg_hams.append(sum(hamming_scores) / len(hamming_scores))

ham_c_df[c] = avg_hams
exact_c_df[c] = avg_exact

print('Best Cross Validated Hamming Scores: ')
print(f'C={penalties[np.nanargmin(ham_c_df.to_numpy())]}; Hamming loss: {ham_c_df[penalties[np.nanargmin(ham_c_df.to_numpy())]].to_numpy()[0]}')
print('\nCross Validated Exact Match Scores: ')
print(f'C={penalties[np.nanargmax(exact_c_df.to_numpy())]}; Exact Match Score: {exact_c_df[penalties[np.nanargmax(exact_c_df.to_numpy())]].to_numpy()[0]}')



Best Cross Validated Hamming Scores: 
C=10000; Hamming loss: 0.048376408217362485

Cross Validated Exact Match Scores: 
C=10000; Exact Match Score: 0.9184890656063618


### iii. <ins>by use SMOTE or any other method you know to remedy class imbalance and retrain SVM. Report your conclusions about the classifiers you trained</ins>

In [9]:
# X_train_res_fam, y_train_fam = SMOTE(random_state=49, n_jobs=-1).fit_resample(X_train_cv, y_train_cv['Family'])
# X_train_res_genus, y_train_genus = SMOTE(random_state=49, n_jobs=-1).fit_resample(X_train_cv, y_train_cv['Genus'])
# X_train_res_genus, y_train_genus = SMOTE(random_state=49, n_jobs=-1).fit_resample(X_train_cv, y_train_cv['Species'])



In [10]:
penalties = [10, 100, 1000, 10000] # penalty term scalars
ham_c_df = pd.DataFrame(columns=penalties) # holds hamming loss for each gamma, C pair
exact_c_df = pd.DataFrame(columns=penalties) # holds exact match score for each gamma, C pair
kf = KFold(n_splits=10, random_state= 49, shuffle=True)

for c in penalties: # test indicated penalty scalars
    avg_hams = []
    avg_exact = []
        
    # 10-fold CV
    for train_index, test_index in kf.split(X_train):
        X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

        hamming_scores = []
        exact_scores = []
        pred_df = pd.DataFrame(columns=y_train.columns, index=y_test_cv.index)

        # train SVM model for each output-label-column
        for col in y_train.columns:
            X_train_res, y_train_res = SMOTE(random_state=49, n_jobs=-1).fit_resample(X_train_cv, y_train_cv[col])
            pipe = Pipeline(steps = [('scale', StandardScaler()),
                                    ('svm', LinearSVC(penalty='l1', multi_class='ovr', max_iter=100000,
                                                      C=c, loss='squared_hinge', dual=False))])
            ovr = pipe.fit(X_train_res, y_train_res)

            pred_df[col] = ovr.predict(X_test_cv)

        # calculate hamming loss
        for i in y_test_cv.index:
            ham_score = 0
            if pred_df['Family'][i] == y_test_cv['Family'][i]:
                ham_score += 1
            if pred_df['Genus'][i] == y_test_cv['Genus'][i]:
                ham_score += 1
            if pred_df['Species'][i] == y_test_cv['Species'][i]:
                ham_score += 1

            exact_scores.append(1 if ham_score == 3 else 0)
            hamming_scores.append(1 - (ham_score / 3))

    avg_exact.append(sum(exact_scores) / len(exact_scores))
    avg_hams.append(sum(hamming_scores) / len(hamming_scores))

ham_c_df[c] = avg_hams
exact_c_df[c] = avg_exact

print('Best Cross Validated Hamming Scores: ')
print(f'C={penalties[np.nanargmin(ham_c_df.to_numpy())]}; Hamming loss: {ham_c_df[penalties[np.nanargmin(ham_c_df.to_numpy())]].to_numpy()[0]}ham_c_gamma_df')
print('\nCross Validated Exact Match Scores: ')
print(f'C={penalties[np.nanargmax(exact_c_df.to_numpy())]}; Exact Match Score: {exact_c_df[penalties[np.nanargmax(exact_c_df.to_numpy())]].to_numpy()[0]}ham_c_gamma_df')



Best Cross Validated Hamming Scores: 
C=10000; Hamming loss: 0.06825712392312791ham_c_gamma_df

Cross Validated Exact Match Scores: 
C=10000; Exact Match Score: 0.8628230616302187ham_c_gamma_df


# 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set
## (a)

In [26]:
from sklearn.metrics import silhouette_score
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html
# https://medium.com/analytics-vidhya/how-to-determine-the-optimal-k-for-k-means-708505d204eb
# https://math.ryerson.ca/~danziger/professor/MTH108/Handouts/codes.pdf
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

ham_distances = []
ham_scores = []
ham_losses = []

for simulation in range(50):
    
    # (a) choose best K automatically by sillhoutte score
    silhouette_scores = []

    for k in range(2, 51):
        km = KMeans(n_clusters = k, random_state=simulation).fit(X)
        labels = km.labels_
        silhouette_scores.append(silhouette_score(X, labels))

    best_k = np.argmax(silhouette_scores) + 2


    # (b) In each cluster, determine which class is the majority by reading the true labels for family, genus, species
    km = KMeans(n_clusters=best_k, random_state=simulation).fit(X)

    clusters = km.labels_
    family_labels  = y['Family'].unique()
    genus_labels   = y['Genus'].unique()
    species_labels = y['Species'].unique()

    col_to_label_to_freq = {'Family': dict(zip(family_labels,  len(family_labels)*[0]) ), 
                           'Genus':   dict(zip(genus_labels,   len(genus_labels)*[0]) ),
                           'Species': dict(zip(species_labels, len(species_labels)*[0]) )}

    cluster_to_col_to_majClass = {}
    cluster_to_col_to_label_to_freq = {}
    for i in range(0, best_k):
        cluster_to_col_to_label_to_freq[i] = col_to_label_to_freq
        cluster_to_col_to_majClass[i] = dict(zip(['Family', 'Genus', 'Species'], ['', '', '']))

    for i, cluster in enumerate(clusters):
        cluster_to_col_to_label_to_freq[cluster]['Family'][y['Family'][i]] += 1
        cluster_to_col_to_label_to_freq[cluster]['Genus'][y['Genus'][i]] += 1
        cluster_to_col_to_label_to_freq[cluster]['Species'][y['Species'][i]] += 1

    for cluster, column_to_label_to_freq in cluster_to_col_to_label_to_freq.items():
        for col, label_to_freq in column_to_label_to_freq.items():
            
            labels = list(label_to_freq.keys())
            frequencies = list(label_to_freq.values())

            majority_class = labels[frequencies.index(max(frequencies))]
            cluster_to_col_to_majClass[cluster][col] = majority_class

    # (c)
    tot_ham_distance, tot_ham_score, tot_ham_loss = 0, 0, 0
    for i, cluster in enumerate(clusters):
        ham = 0
        if cluster_to_col_to_majClass[cluster]['Family'] == y['Family'][i]:
            ham += 1
        if cluster_to_col_to_majClass[cluster]['Genus'] == y['Genus'][i]:
            ham += 1
        if cluster_to_col_to_majClass[cluster]['Species'] == y['Species'][i]:
            ham += 1

        tot_ham_distance += (3 - ham)
        tot_ham_score += (ham / 3)
        tot_ham_loss += (1 - (ham/3))


    avg_ham_distance = tot_ham_distance / len(clusters)
    avg_ham_score = tot_ham_score / len(clusters)
    avg_ham_loss = tot_ham_loss / len(clusters)

    ham_distances.append(avg_ham_distance)
    ham_scores.append(avg_ham_score)
    ham_losses.append(avg_ham_loss)

print(f'Hamming Score-- avg: {np.average(ham_scores)}; std. dev.: {np.std(ham_scores)}')
print(f'Hamming Loss-- avg: {np.average(ham_losses)}; std. dev.: {np.std(ham_losses)}')
print(f'Hamming Distance-- avg: {np.average(ham_distances)}; std. dev.: {np.std(ham_distances)}')

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


KeyboardInterrupt: 

In [22]:
print(len(km.labels_))
print(sum(km.labels_))

7195
8622


# 3
### <ins>ISLR: 12.6.2</ins>
Suppose that we have four observations, for which we compute a dissimilarity matrix, given by:

$$\begin{pmatrix}& .3 & .4 & .7\\.3 & & .5 & .8\\.4 & .5 & & .45\\.7 & .8 & .45\end{pmatrix}$$

For instance, the dissimilarity between the first and second observations is 0.3, and the dissimilarity between the second and fourth observations is 0.8.

(a) On the basis of this dissimilarity matrix, sketch the dendrogram that results from hierarchically clustering these four observa- tions using complete linkage. Be sure to indicate on the plot the height at which each fusion occurs, as well as the observations corresponding to each leaf in the dendrogram.
- answer

(b) Repeat (a), this time using single linkage clustering.
- answer

(c) Suppose that we cut the dendrogram obtained in (a) such that two clusters result. Which observations are in each cluster?
- answer

(d) Suppose that we cut the dendrogram obtained in (b) such that two clusters result. Which observations are in each cluster?
- answer

(e) It is mentioned in the chapter that at each fusion in the dendrogram, the position of the two clusters being fused can be swapped without changing the meaning of the dendrogram. Draw a dendrogram that is equivalent to the dendrogram in (a), for which two or more of the leaves are repositioned, but for which the meaning of the dendrogram is the same.
- answer
